# Diffie-Hellman & RSA Key Exchange 정리 및 실습

공무원 시험 및 보안기사 필기 대비 실무자용 Python 실습 노트북입니다.

## Diffie-Hellman Key Exchange

1. 공개소수 `p`, 원시근 `g`를 공유한다.
2. Alice는 비밀키 `a`, Bob은 비밀키 `b`를 고른다.
3. Alice는 공개키 `A = g^a mod p`, Bob은 `B = g^b mod p` 계산 후 교환
4. 공유 비밀키: `K = B^a mod p = A^b mod p`


In [4]:
from asymmetric_key.rsa import signature


# Diffie-Hellman 구현 예제
def diffie_hellman(p, g, a, b):
    A = pow(g, a, p)
    B = pow(g, b, p)
    shared_key_alice = pow(B, a, p)
    shared_key_bob = pow(A, b, p)
    return A, B, shared_key_alice, shared_key_bob

# 예시
p = 23  # 소수
g = 7   # 원시근
a = 3   # Alice의 비밀키
b = 2  # Bob의 비밀키

A, B, key_alice, key_bob = diffie_hellman(p, g, a, b)
print("Alice의 공개키 A:", A)
print("Bob의 공개키 B:", B)
print("Alice가 계산한 공유키:", key_alice)
print("Bob이 계산한 공유키:", key_bob)

Alice의 공개키 A: 21
Bob의 공개키 B: 3
Alice가 계산한 공유키: 4
Bob이 계산한 공유키: 4


## ✅ Diffie-Hellman 구조 요약 (시험 대비용 실전 정리)

### 1. 개념 흐름

> 공개된 값 p(소수), g(원시근)는 모두 공유
>

| 주체 | 비밀키 | 공개키 | 공유 비밀 |
| --- | --- | --- | --- |
| Alice | a | A = g^a mod p | K = B^a mod p |
| Bob | b | B = g^b mod p | K = A^b mod p |

→ 결국 공유 비밀 K는 동일함: `K = g^(ab) mod p`

> 💡 이 공유 비밀은 **대칭키(세션키)**로 사용됨. 암호화는 이걸로.
>

---

### 2. 실제 시험형 예제

> p=23, g=5, a=6, b=15 이면?
>
- Alice: A = 5^6 mod 23 = 15625 mod 23 = 8
- Bob: B = 5^15 mod 23 = ... = 2
- 공유 키: K = B^a mod 23 = 2^6 mod 23 = 64 mod 23 = **18**

(혹은 K = A^b mod 23도 동일)

---

### 3. 공무원 필기에서 나올 수 있는 문제 패턴

- `p`, `g`, `a`, `b` 중 3개 주고 공유 비밀 키 구하라
- `g`, `a`, `p` 주고 공개키 A를 구하라 (`g^a mod p`)
- 지수연산 반복적으로 쓰는 연산자 의미 묻기 (`modular exponentiation`)
- RSA와 비교 구조 물어보기: **DH는 키 합의**, **RSA는 직접 암복호화**

---

## 📘 실전용 공식 요약 카드

```
# Diffie-Hellman 요약 카드

[공개] 소수 p, 원시근 g
[개인] Alice: a, Bob: b
[공개키] A = g^a mod p, B = g^b mod p
[공유키] K = A^b mod p = B^a mod p = g^(ab) mod p
[사용처] 공유된 비밀키로 대칭키(세션키) 생성

```

## 🔐 Diffie-Hellman 핵심 개념 정리 (공무원 시험용 용어 구분)

| 항목 | 설명 | 예시 변수 | 생성 방법 | 용도 |
| --- | --- | --- | --- | --- |
| ✅ **개인키** | 각자만 알고 있는 **난수** | `a`, `b` | 무작위 난수 | 공유 비밀키 생성에 사용 |
| ✅ **공개키** | 상대에게 전달되는 값 | `A = g^a mod p`, `B = g^b mod p` | 개인키로 계산 | 상대가 이걸로 공유키 계산 |
| ✅ **공유 비밀키** | Alice와 Bob이 **각자 계산하지만 동일한 값** | `K = B^a mod p = A^b mod p = g^(ab) mod p` | 서로의 공개키와 자신의 개인키로 계산 | **세션키의 재료** |
| ✅ **세션키** | 공유 비밀키 `K`를 바탕으로 만든 대칭키 | 예: `AES_key = SHA256(K)` | 공유키에 대해 해시함수 등으로 파생 | 암호화 실제 사용 키 |

---

### 🔁 흐름 정리

1. **서로 다른 난수 `a`, `b` 생성** → 개인키
2. `A = g^a mod p`, `B = g^b mod p` → 공개키 서로 교환
3. 각자 **`K = g^(ab) mod p` 계산** → 공유 비밀키
4. `K`로부터 SHA256 등 해시 → 실제로 쓰는 **세션키**

---

### 📌 핵심 요약 카드

```
- 개인키: 난수 (a, b)
- 공개키: g^a mod p, g^b mod p
- 공유 비밀키: g^(ab) mod p (같은 값)
- 세션키: 공유 비밀키로부터 파생된 대칭키 (AES에 사용)

```

---

### 💡 시험 대비 포인트

| 문제 유형 | 요점 |
| --- | --- |
| `p, g, a, b` 주고 공유키 구하라 | `g^(ab) mod p` 계산 |
| 공개키만 보고 `K` 구하라 | 상대 공개키 ^ 내 개인키 |
| 세션키 관련 보기 | **K는 암호화에 직접 사용되지 않고, 세션키의 재료**라는 점 강조 |

## 🔥 원시근(primitive root)이란?

간단히 말해:

> 어떤 소수 p에 대해, 그 원시근 g는
>
>
> **g^1, g^2, ..., g^(p-1)** mod p 를 계산했을 때,
>
> **1부터 p−1까지의 모든 수가 전부 나오는 g**
>

---

### ✅ 예시로 이해

### 소수 `p = 7`일 때,

`g = 3`을 기준으로:

```
3^1 mod 7 = 3
3^2 mod 7 = 2
3^3 mod 7 = 6
3^4 mod 7 = 4
3^5 mod 7 = 5
3^6 mod 7 = 1
```

→ 나온 결과: `{1, 2, 3, 4, 5, 6}`

→ **1부터 6까지 전부 다 나왔죠?**

➡️ **그래서 3은 7의 원시근입니다.**

---

### 📘 정의 요약

```
소수 p에 대해,
g^k mod p (1 ≤ k ≤ p−1) 이 p의 모든 잔여 클래스를 만들어내면,
g를 p의 원시근이라 한다.
```

---

## ❓ 왜 필요한가요?

Diffie-Hellman은 공유 비밀키 `g^(ab) mod p` 를 생성하죠.

- 이때 `g`가 원시근이면, `g^x mod p`의 결과가 **무작위처럼 잘 퍼집니다**
- 그래서 `g^a mod p`, `g^b mod p`가 진짜 난수처럼 보이고 **보안성↑**
- 반면, `g`가 원시근이 아니면 일부 값만 나오기 때문에 **공격자에게 예측 가능성↑**

➡️ **원시근은 Diffie-Hellman 보안성을 보장하는 수학적 뿌리**

---

## 🧪 원시근 찾는 Python 코드 예시

In [2]:
def is_primitive_root(g, p):
    required_set = set(range(1, p))
    actual_set = set(pow(g, power, p) for power in range(1, p))
    return required_set == actual_set

# 예시
p = 7
for g in range(2, p):
    if is_primitive_root(g, p):
        print(f"{g}는 {p}의 원시근")


3는 7의 원시근
5는 7의 원시근


## ✅ 시험 대비 핵심 요약

| 항목 | 설명 |
| --- | --- |
| 정의 | `g^k mod p`가 1~p−1 모두 만들면 원시근 |
| 이유 | 공유 비밀키가 모든 값을 가질 수 있게 하는 근거 |
| 확인 | 원시근이면 `g^1 ~ g^(p−1)`이 전부 다 나옴 |
| 시험 팁 | p 작으면 직접 계산 가능 (`p = 7` 정도) |

## ✅ 먼저 개념을 딱 구분하자:

| 위험요소 | 의미 | 보안영향 |
| --- | --- | --- |
| 💀 **중간자 공격** | 키 교환 중 **공개키 전달을 가로채서** 변조 | **가짜 키를 강제로 공유하게 됨** |
| ⚠️ **원시근 아님** | `g^x mod p` 값이 일부 값만 나와 **예측 가능** | 키 범위가 줄어들어 **추측하기 쉬움** |

---

## 💀 중간자 공격(MITM) - 어떻게 이루어지나?

### 상황:

1. Alice → Bob 에게 `A = g^a mod p` 전송
2. 공격자 Mallory가 중간에서 가로채고 자신만의 `M = g^m mod p` 보내버림
3. Bob은 M을 진짜 A인 줄 알고, `K = M^b mod p` 계산
4. 반대로, Mallory는 A와 B 모두 알고 있으므로:
    - Mallory: `K_A = A^m mod p`, `K_B = B^m mod p`
    - → **각자와 다른 공유 비밀을 가지지만 몰래 내용을 복호화 가능**

즉, Alice–Bob 사이엔 **진짜 공유키가 없고**, 둘 다 **Mallory와만 공유함**

➡️ **암호화된 메시지를 Mallory가 몰래 보고, 심지어 조작도 가능**

---

## 🔐 이걸 막으려면?

- **DH는 본질적으로 인증이 없습니다** → 그래서 **TLS에선 반드시 인증서/서명 사용**
- 공무원 시험 기준:
    - **DH 자체는 MITM 공격에 취약하다** (✔ 정답 보기)
    - **전자서명이나 공개키 인증서 등으로 막아야 한다**

---

## ⚠️ 원시근이 아닐 때는?

- 공개된 `g`가 **원시근이 아니면**,

    → `g^x mod p`의 결과 값이 **p−1개 전부가 아니라 일부만 나옴**

    → 공유 비밀키 `K`도 **그 일부 값만 가능**해짐

    → 공격자가 **모든 가능한 K 후보를 다 시도하는 공격** 가능


➡️ **이건 MITM처럼 바꾸는 게 아니라, 추측(brute force) 공격이 쉬워지는 문제**

---

## 📘 비교 요약표

| 항목 | 중간자 공격(MITM) | 원시근 X 문제 |
| --- | --- | --- |
| 공격 방식 | 공개키 교환 중 가로채기 | 공유키 후보 수가 줄어 추측 가능 |
| 예시 | Mallory가 Alice–Bob 중간에 개입 | g=4인데 실제 생성 가능한 값이 5개뿐 |
| 방어법 | 인증서, 서명 등으로 인증 | 반드시 원시근 사용 (`g` 검증) |
| 시험문제 포인트 | **Diffie-Hellman은 MITM에 취약하다** | **원시근이 아니면 키공간 작아진다** |

---

## ✅ 공무원 시험 대비 요점 정리

- Diffie-Hellman은 본질적으로 **MITM 공격에 취약** → 서명·인증 필요
- 원시근을 사용하지 않으면 **공유 비밀키가 충분히 무작위가 아니다**
- 두 개념을 **섞지 말고 정확히 구분해서 기억**하세요

## RSA Key Generation & Encryption/Decryption

1. 소수 `p`, `q` 선택
2. `n = p * q`, `phi = (p-1)*(q-1)` 계산
3. `1 < e < phi` 이면서 서로소인 `e` 선택
4. `d ≡ e⁻¹ mod phi` (e의 모듈러 역원) 계산
5. 공개키 `(e, n)`, 개인키 `(d, n)`
6. 암호화: `C = M^e mod n`, 복호화: `M = C^d mod n`


In [2]:
# RSA 구현 예제
from sympy import mod_inverse, isprime, gcd

def rsa_keygen(p, q, e):
    assert isprime(p) and isprime(q), "p, q는 소수여야 함"
    n = p * q
    phi = (p - 1) * (q - 1)
    assert gcd(e, phi) == 1, "e와 phi는 서로소여야 함"
    d = mod_inverse(e, phi)
    return (e, n), (d, n)

def rsa_encrypt(m, pubkey):
    e, n = pubkey
    return pow(m, e, n)

def rsa_decrypt(c, privkey):
    d, n = privkey
    return pow(c, d, n)

# 예시
p, q = 61, 53
e = 17
pubkey, privkey = rsa_keygen(p, q, e)
message = 65

cipher = rsa_encrypt(message, pubkey)
print("암호문:", cipher)
plain = rsa_decrypt(cipher, privkey)
print("복호화된 평문:", plain)

암호문: 2790
복호화된 평문: 65


## ECC (Elliptic Curve Cryptography) 개념 및 실습

- 타원곡선 위의 덧셈/곱셈 연산 사용
- E: y² = x³ + ax + b (mod p) 형태의 곡선 위의 점들을 이용
- 개인키: 정수 `k`, 공개키: `R = kP` (P는 공개된 곡선 위 점)
- 문제: `P`, `R`가 주어졌을 때 `k`를 찾는 것 → 이산로그 문제

👉 공무원/보안기사 시험에서 나오는 유형: `R = kP` 만족하는 `k`를 그래프 보고 찾기 (시각적 순환 구조)


In [3]:
# ECC 곡선 상의 점 덧셈 구현 (단순한 예시, 실제는 더 복잡함)
# y^2 = x^3 + ax + b over finite field mod p
def inverse_mod(k, p):
    return pow(k, -1, p)

def ecc_add(P, Q, a, p):
    if P == Q:
        l = (3 * P[0]**2 + a) * inverse_mod(2 * P[1], p)
    else:
        l = (Q[1] - P[1]) * inverse_mod(Q[0] - P[0], p)
    l %= p
    x_r = (l**2 - P[0] - Q[0]) % p
    y_r = (l * (P[0] - x_r) - P[1]) % p
    return (x_r, y_r)

def ecc_mul(P, k, a, p):
    R = P
    for _ in range(k - 1):
        R = ecc_add(R, P, a, p)
    return R

# 예시: E: y^2 = x^3 + 2x + 2 mod 17
a, b, p = 2, 2, 17
P = (5, 1)
k = 7
R = ecc_mul(P, k, a, p)
print(f"k = {k}일 때 R = kP = {R}")

k = 7일 때 R = kP = (0, 6)


### ✅ 실무자 시점에서 ECC 감 잡는 팁

- **비트크기 비교로 접근**하세요:
    - RSA 2048 ≈ ECC 224~256
- *공개키 = 개인키 × G (점 배수 연산)**만 기억하면 됨
- Diffie-Hellman처럼 생각하되, 연산 대상이 "정수"가 아니라 "곡선 위의 점"이라는 차이
- 파이썬에서 `ecdsa` 패키지 써보면 금방 감이 옵니다

### 🔹 `ecdsa` 패키지란?

- **타원 곡선 기반 공개키 암호 시스템**을 구현할 수 있는 파이썬 라이브러리입니다.
- 특히 **ECDSA (Elliptic Curve Digital Signature Algorithm)**을 사용하여 **전자서명**, **키 생성**, **서명 검증** 등의 작업을 할 수 있습니다.
- 실무에서는 **서명(Signing)**, **검증(Verifying)** 기능을 구현할 때 사용합니다.

In [23]:
from ecdsa import SigningKey, SECP256k1

sk = SigningKey.generate(curve=SECP256k1)
vk = sk.verifying_key
print("개인키: ", sk.to_string().hex())
print("공개키: ", vk.to_string().hex())

개인키:  fec79a5e4d5738903ad796589074fe69d6e39abeaa907c6cc6f3ca9c51a81266
공개키:  ea8dc1a65aea2b122cfcbb252ea6b76aa900a7a4e81d68b6d46d0ed54f50fa41001c7180a1ddacedb6c4676e1129eb64b4bb8735cdcf0cff9bac971c08468efe


In [24]:
message = b"cheerUp!!"
signature = sk.sign(message)

is_valid = vk.verify(signature, message)
print("서명 유효 여부: ", is_valid)

서명 유효 여부:  True


### 🔹 어떤 타원곡선 지원하나요?

- `SECP256k1`: 비트코인에서 사용하는 곡선
- `NIST192p`, `NIST256p` 등도 지원

---

### 🔹 공무원 시험과 연관성

- 공무원 시험에서는 `ECDSA`가 아래처럼 보기로 출제될 수 있습니다:

```
다음 중 디지털 서명 알고리즘이 아닌 것은?
① RSA
② DSA
③ ECDSA
④ AES  ← 정답
```

---

### 🔹 참고

- ECDSA는 **DSA의 타원곡선 버전**입니다.
- ECC 기반 디지털 서명이므로 **공개키 = 개인키 × G** 형태로 연산됩니다.
- 키 길이가 짧으면서도 보안성이 높아 스마트카드, IoT, 비트코인 등에 많이 사용됩니다.

---

### 🧩 예시 문제

타원곡선 암호시스템(ECC)은 타원곡선 이산대수의 어려움을 이용한다. 그림ㄴ과 같이 실수 위에 정의된 타원곡선과 타원곡선 상의 두 점 P와 R 이 주어진 경우 R = kP를 만족하는 정수 k의 값은?

> 보기: 2, 3, 4, 5

#### 문제 요약

> R = kP가 성립하는 정수 k를 구하라
>
> - P: 타원곡선 위의 점
> - R: 또 다른 타원곡선 위의 점
> - 연산: 점 덧셈(`kP = P + P + ... + P`, 총 k번)

---

### 🧠 이건 "타원곡선 이산로그 문제 (ECDLP)"입니다.

- 일반적으로 어려운 문제지만,
- **곡선이 작고 점 개수가 적으면 bruteforce(완전탐색)** 가능합니다.

In [32]:
from ecdsa.ellipticcurve import Point, CurveFp

# 곡선 정의 예시: y^2 = x^3 + 2x + 2 over F_17
p = 17
a = 2
b = 2
curve = CurveFp(p, a, b)

# 문제에서 주어진 점들 (예시값, 문제의 그림이 없으니 임의로 설정)
P = Point(curve, 5, 1)  # Generator point P
R = Point(curve, 6, 3)  # R = kP 라고 가정

# 보기 후보 (시험에서는 이렇게 주어짐)
choices = [2, 3, 4, 5]

# 후보 중에서 일치하는 k 찾기
for k in choices:
    result = P * k  # 점의 배수 연산
    print(f"{k}P = ({result.x()}, {result.y()})")
    if result == R:
        print(f"코드상 정답: k = {k}")
        break
else:
    print("보기 중에는 정답이 없음")


2P = (6, 3)
코드상 정답: k = 2


### ⚠️ 주의할 점

1. **곡선이 유한체(mod p)에서 정의되어야 함**
2. `ecdsa.ellipticcurve`는 **실험용**이며, 성능 최적화 안 되어 있음
3. 큰 곡선에서는 절대 bruteforce 불가 (보안성의 핵심이기도 함)

---

### 🔍 실무자 시점 요약

- `R = kP`는 **ECDLP 문제**
- 실무에서는 brute-force 대신 **Shank’s baby-step giant-step** 등의 알고리즘 필요
- 시험에서는 대부분 "작은 점에서 k 구하기"처럼 출제 → brute force 충분

---

### ✍️ 공무원 시험 정리용 표현

> 타원곡선 암호는 타원곡선 위의 두 점 P, R에 대해
>
>
> `R = kP`를 만족하는 정수 k를 찾기 어려운 문제(ECDLP)에 기반한다.
>

---

이 `p = 17`, `a = 2`, `b = 2`는 **타원곡선 암호의 예제용 파라미터**로,

**곡선의 수학적 정의를 위해 사람이 임의로 지정한 값**이에요.

---

### ✅ 이 값들의 의미

타원곡선 암호(ECC)에서 사용되는 곡선은 다음과 같은 형태를 가집니다:


$y^2 \equiv x^3 + ax + b \mod p$


여기서:

| 기호 | 의미 |
| --- | --- |
| `a`, `b` | 타원곡선을 정의하는 상수 (곡선의 모양 결정) |
| `p` | 유한체(Finite Field)의 소수 모듈러 값 |
| 전체 | **Fp 위의 타원곡선**을 정의함 |

---

### ✅ `p=17, a=2, b=2`는 왜 사용되나요?

- **`p=17`**: 작은 소수라서 수동 계산 가능 (시험용, 예제용에 적합)
- **`a=2`, `b=2`**: 타원곡선 조건을 만족하면서 계산이 간단한 값
- 이 값들을 사용하면 **곡선 위에 존재하는 점들**을 수동으로 구하거나

    **`kP` 계산을 시뮬레이션하기에 쉬워서** 자주 쓰입니다.


---

### ✅ 실전에서는 어떤 값을 쓰나요?

| 용도 | 파라미터 |
| --- | --- |
| 시험/학습용 | 작은 p, 작고 단순한 a, b (ex. 17, 2, 2) |
| 실무/보안 | 256비트 이상 큰 p, 보안 기준에 따라 정해진 a, b |
| 표준곡선 | NIST, SECG 등이 정의한 `secp256k1`, `NIST P-256` 등 |

---

### ✅ 타원곡선 조건 확인 (시험 포인트)

타원곡선이 유효하려면 다음 조건을 만족해야 합니다:

$4a^3 + 27b^2 \not\equiv 0 \mod p$


예를 들어,

```python
a = 2
b = 2
p = 17

Δ = 4 * a**3 + 27 * b**2
print(Δ % p)  # ≠ 0 이면 유효한 곡선
```

---

### 🔍 결론

- `p = 17, a = 2, b = 2`는 **예제용으로 사람이 정한 값**입니다.
- 보안이 아닌, **직관적 이해와 계산 연습을 위한 용도**예요.
- 실전에서는 표준 타원곡선(`secp256k1` 등)을 사용합니다.

In [4]:
# ElGamal 이산로그 문제
def discrete_log_brute(a, y, p):
    for x in range(1, p):
        if pow(a, x, p) == y:
            return x
    return None

# 예시
p = 23
a = 5
y = 4
x = discrete_log_brute(a, y, p)
print(f"a = {a}, y = {y}, p = {p}일 때, a^x ≡ y mod p 를 만족하는 x = {x}")

a = 5, y = 4, p = 23일 때, a^x ≡ y mod p 를 만족하는 x = 4
